In [1]:
from typing import Dict, List
import os
from datetime import datetime, timedelta

import plotly.graph_objects as go
import plotly.express as px

def timestr_to_seconds(time_str: str) -> float:
    if 'day' in time_str:
        fmt = '%d day, %H:%M:%S.%f'
        time = datetime.strptime(time_str, fmt) + timedelta(days=1)
    else:
        fmt = '%H:%M:%S.%f'
        time = datetime.strptime(time_str, fmt)
    return (time - datetime(1900, 1, 1)).total_seconds()

def read_log(log_path: str) -> Dict[str, List[float | int]]:

    iters: List[int] = []
    lasted: List[float] = []
    best_costs: List[int] = []

    with open(log_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('wandb:'):
                continue
            elif line.startswith('Data for iter '):
                i_iter = int(line.split('Data for iter ')[1].split(' ')[0])
                iters.append(i_iter)
            elif line.startswith('Training lasted '):
                #   Training lasted 1 day, 0:01:26.081946 .
                time_str = line.split('Training lasted ')[-1][:-2]
                secs = timestr_to_seconds(time_str)
                lasted.append(secs)
            elif '_best_graph_cost' in line:
                cost = int(line.split(' ')[-1])
                best_costs.append(cost)
    
    data = {
        'iteration': iters,
        'time': lasted,
        'best cost': best_costs,
    }
    return data


def draw_figure(
    X: List[float | int], Ys: Dict[str, List[float | int]],
    title: str | None, xaxis_title: str, yaxis_title: str,
) -> go.Figure:
    fig = go.Figure()
    for y_name, y in Ys.items():
        fig.add_trace(go.Scatter(
            x=X, y=y, mode='lines', name=y_name,
        ))
    # end for
    fig.update_layout(
        title={
            'text': title,
            'x': 0.5,
        } if title else None,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        # margin=dict(l=0,r=0,b=0,t=0),
    )
    fig.update_layout(
        legend=dict(
            yanchor="top", y=0.99,
            xanchor="right", x=0.99,
        ),
    )
    # fig.update_xaxes(showgrid=False, showline=True, mirror=True, linewidth=1, linecolor='black')
    # fig.update_yaxes(showgrid=False, showline=True, mirror=True, linewidth=1, linecolor='black')
    # fig.update_layout({
    #     'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    # })
    return fig

def draw_figure_pair(
    XYs: Dict[str, List[List[float | int]]],
    title: str | None, xaxis_title: str, yaxis_title: str,
) -> go.Figure:
    fig = go.Figure()
    for name, (X, Y) in XYs.items():
        fig.add_trace(go.Scatter(
            x=X, y=Y, mode='lines', name=name,
        ))
    # end for
    fig.update_layout(
        title={
            'text': title,
            'x': 0.5,
        } if title else None,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        # margin=dict(l=0,r=0,b=0,t=0),
        font=dict(size=16, color='black'),
    )
    fig.update_layout(
        legend=dict(
            yanchor="top", y=0.99,
            xanchor="right", x=0.99,
        ),
    )
    fig.update_xaxes(showgrid=False, showline=True, mirror=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showgrid=False, showline=True, mirror=True, linewidth=1, linecolor='black')
    fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    })
    return fig


In [ ]:
log_w_pret = read_log('nam_adder_8_w_pret.log')
log_wo_pret = read_log('nam_adder_8_wo_pret.log')

fig = draw_figure_pair(
    {
        'w/ pretraining': [log_w_pret['time'], log_w_pret['best cost']],
        'w/o pretraining': [log_wo_pret['time'], log_wo_pret['best cost']],
    },
    None, 'Time (sec)', 'Minimum Gate Count',
)
fig.show()
fig.write_image(f'adder_8_comp_pret.pdf', width=700, height=300)

In [55]:
log_4 = read_log('nam_4_gf2^6_mult.log')
log_6 = read_log('nam_6_gf2^6_mult.log')
log_8 = read_log('nam_8_gf2^6_mult.log')
log_10 = read_log('nam_10_gf2^6_mult.log')

max_iteration = max(len(log_6['iteration']), len(log_8['iteration']), len(log_10['iteration']))

fig = draw_figure_pair(
    {
        'k = 4': [log_4['iteration'], log_4['best cost']],
        'k = 6': [log_6['iteration'], log_6['best cost']],
        'k = 8': [log_8['iteration'], log_8['best cost']],
        'k = 10': [log_10['iteration'], log_10['best cost']],
    },
    'Best cost - iteration', 'iteration', 'best cost',
)

fig.show()